In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geohash

In [39]:
featuredDataset = pd.read_csv('featured-dataset.csv')
featuredDataset = featuredDataset.drop(featuredDataset.columns[0], axis=1)
featuredDataset.head(5)


,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,...,z_start,location_start,location_end,linea_org,tipo_usuario,ruta_org,viaje_org,coche_org,nodo_org,nodo_dest
0,2022,11,7,5.3:18.0,0.254167,-0.026177,0.999657,Monday,0.036310,0.974089,...,-0.500105,eztr0z7h,eztr38u0,2,41,1,2,2,509,76
1,2022,11,7,5.3:18.0,0.254167,-0.026177,0.999657,Monday,0.036310,0.974089,...,-0.500105,eztr0z7h,eztr3263,2,41,1,2,2,509,41
2,2022,11,7,5.766666666666667:46.0,0.293056,-0.267238,0.963630,Monday,0.041865,0.965602,...,-0.494151,eztr3bdk,eztr20bt,16,41,2,3,3,36,403
3,2022,11,7,5.9:54.0,0.304167,-0.333807,0.942641,Monday,0.043452,0.962961,...,-0.495661,eztr322d,eztr3053,2,41,2,3,3,42,11
4,2022,11,7,5.983333333333333:59.0,0.311111,-0.374607,0.927184,Monday,0.044444,0.961262,...,-0.500003,eztr0z6v,eztr39e9,2,41,2,3,3,512,137


In [40]:
featuredDataset.shape

(92727, 24)

### Feature extraction

In [41]:

# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) >= 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0
    
def further_data_prep(df):  

    df['start_lat'] = df['location_start'].apply(lambda geo: decodegeo(geo, 0))
    df['start_lon'] = df['location_start'].apply(lambda geo: decodegeo(geo, 1))
    df['end_lat'] = df['location_end'].apply(lambda geo: decodegeo(geo, 0))
    df['end_lon'] = df['location_end'].apply(lambda geo: decodegeo(geo, 1))
    
    return df

In [42]:
featuredDataset = further_data_prep(featuredDataset)
featuredDataset.head(5)

,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,...,tipo_usuario,ruta_org,viaje_org,coche_org,nodo_org,nodo_dest,start_lat,start_lon,end_lat,end_lon
0,2022,11,7,5.3:18.0,0.254167,-0.026177,0.999657,Monday,0.036310,0.974089,...,41,1,2,2,509,76,43.458567,-3.829937,43.466120,-3.795605
1,2022,11,7,5.3:18.0,0.254167,-0.026177,0.999657,Monday,0.036310,0.974089,...,41,1,2,2,509,41,43.458567,-3.829937,43.463545,-3.808994
2,2022,11,7,5.766666666666667:46.0,0.293056,-0.267238,0.963630,Monday,0.041865,0.965602,...,41,2,3,3,36,403,43.465433,-3.787022,43.466978,-3.866329
3,2022,11,7,5.9:54.0,0.304167,-0.333807,0.942641,Monday,0.043452,0.962961,...,41,2,3,3,42,11,43.463717,-3.811398,43.462172,-3.818607
4,2022,11,7,5.983333333333333:59.0,0.311111,-0.374607,0.927184,Monday,0.044444,0.961262,...,41,2,3,3,512,137,43.458738,-3.830280,43.470411,-3.796291


### Train-test split
For Cross Validation, we split the data into 80% train set and 20% test set.

In [43]:
columns_all_features = featuredDataset.columns
columns_X = ['day_num', 'x_start', 'y_start','z_start','linea_org','tipo_usuario','ruta_org','viaje_org','day_num','time_num']
columns_y = ['nodo_dest']
X = featuredDataset[columns_X]
y = featuredDataset[columns_y]

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [45]:
print ('X: ({}, {})'.format(*X.shape))
print ('y: ({}, {})'.format(*y.shape))
print ('X_train: ({}, {})'.format(*X_train.shape))
print ('y_train: ({}, {})'.format(*y_train.shape))
print ('X_test: ({}, {})'.format(*X_test.shape))
print ('y_test: ({}, {})'.format(*y_test.shape))

X: (92727, 10)
y: (92727, 1)
X_train: (88090, 10)
y_train: (88090, 1)
X_test: (4637, 10)
y_test: (4637, 1)


### Machine Learning

In [46]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [47]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=-1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print ("BEST", gs.best_params_, gs.best_score_, gs.cv_results_)
    best = gs.best_estimator_
    return best

#### Grid Search Cross Validation
Grid Search CV implements an exhaustive search over specified parameter values for an estimator.
Important members are fit, predict.

It iterates through a dictionary of hyper parameters and choose the combination that better fits the model.

In [48]:
# Create a k-Nearest Neighbors Regression estimator
knn_estimator = KNeighborsRegressor()
knn_parameters = {"n_neighbors": [1,2,5,10,20,50,100]}
knn_parameters = {"n_neighbors": [8,7,9]}
knn_best = cv_optimize(knn_estimator, knn_parameters, X_train, y_train, score_func='neg_mean_squared_error')

BEST {'n_neighbors': 9} -24153.382284952677 {'mean_fit_time': array([0.16610827, 0.1766305 , 0.16768832]), 'std_fit_time': array([0.03382802, 0.02338799, 0.01131866]), 'mean_score_time': array([1.28688908, 1.20463753, 1.13635397]), 'std_score_time': array([0.01133948, 0.07195307, 0.01406623]), 'param_n_neighbors': masked_array(data=[8, 7, 9],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 8}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([-24283.10431537, -24426.68949312, -24137.09855381]), 'split1_test_score': array([-24322.60156516, -24577.86187943, -24092.71751954]), 'split2_test_score': array([-24197.59540757, -24484.96179232, -24016.59833307]), 'split3_test_score': array([-24611.7213991 , -24926.68652306, -24421.03582826]), 'split4_test_score': array([-24266.27166463, -24547.31140809, -24099.46119008]), 'mean_test_score': array([-24336.25887037, -24592.70221921, -24153.38228495]), 'std_tes

In [49]:
from sklearn.neural_network import MLPClassifier
knn_estimator = MLPClassifier()
knn_parameters = {"n_neighbors": [1,2,5,10,20,50,100]}
knn_parameters = {"n_neighbors": [8,7,9]}
knn_best = cv_optimize(knn_estimator, {}, X_train, y_train, score_func='neg_mean_squared_error')


/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1096: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1096: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/neural_network/_multi

BEST {} -39193.19321148825 {'mean_fit_time': array([776.96566052]), 'std_fit_time': array([60.18862302]), 'mean_score_time': array([0.18436007]), 'std_score_time': array([0.01684426]), 'params': [{}], 'split0_test_score': array([-35810.90140765]), 'split1_test_score': array([-40869.63446475]), 'split2_test_score': array([-41844.68571915]), 'split3_test_score': array([-37725.06867976]), 'split4_test_score': array([-39715.67578613]), 'mean_test_score': array([-39193.19321149]), 'std_test_score': array([2177.57403188]), 'rank_test_score': array([1], dtype=int32)}


/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Model accuracy: R-Squared and Root-mean-squared deviation
R-squared is a statistical measure of how close the data are to the fitted regression line. It ranges from 0 to 1, being 1 the best coefficient.
RMSE is the square root of the mean square error. In other worids the distance, on average, of a data point from the fitted line, measured along a vertical line.

In [ ]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
knn_reg=knn_best.fit(X_train, y_train)
knn_training_accuracy = knn_reg.score(X_train, y_train)
knn_test_accuracy = knn_reg.score(X_test, y_test)
print ("############# based on standard predict ################")
print ("R^2 on training data: %0.8f" % (knn_training_accuracy))
print ("R^2 on test data:     %0.8f" % (knn_test_accuracy))

/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1096: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
# Calculate the Root Mean Squared Error
np.sqrt(mean_squared_error(knn_reg.predict(X_test),y_test))

0.013459698677740432

In [14]:
sampleds = pd.DataFrame(featuredDataset, columns=(columns_X + columns_y))
sampleds = sampleds.sample(10)
sampleds

,day_num,x_start,y_start,linea_org,tipo_usuario,ruta_org,viaje_org,day_num,time_num,end_lat,end_lon
53774,0.533532,-0.640979,0.578246,2,1,1,11,0.533532,0.734722,43.461142,-3.823071
21219,0.517460,-0.676427,0.543638,8,61,1,8,0.517460,0.622222,43.461313,-3.845043
61154,0.925595,-0.673471,0.547450,2,51,1,7,0.925595,0.479167,43.469381,-3.807964
34929,0.070437,-0.686640,0.538204,2,71,1,6,0.070437,0.493056,43.463717,-3.811398
10926,0.250992,-0.672777,0.546842,2,1,1,11,0.250992,0.756944,43.463202,-3.805904
42673,0.240079,-0.679084,0.537977,2,4,1,14,0.240079,0.680556,43.455133,-3.819981
23697,0.628968,-0.660061,0.562432,3,2,2,3,0.628968,0.402778,43.466635,-3.865986
40415,0.211905,-0.674616,0.544277,3,11,1,7,0.211905,0.483333,43.464575,-3.807278
62813,0.056349,-0.700376,0.523332,2,2,2,3,0.056349,0.394444,43.463545,-3.802128
57557,0.654762,-0.698404,0.529245,2,100,2,15,0.654762,0.583333,43.463373,-3.805218


In [15]:
y_pred = knn_reg.predict(sampleds.iloc[:,:-2])
y_pred

array([[43.46249342, -3.82113934],
       [43.46631289, -3.80783558],
       [43.46652746, -3.80410194],
       [43.46369505, -3.82783413],
       [43.4662056 , -3.80178452],
       [43.45837355, -3.81611824],
       [43.46283674, -3.84680271],
       [43.46543312, -3.80350113],
       [43.46493959, -3.80757809],
       [43.46137762, -3.81023884]])

### Save the model
We dump the trained model into a file, so that we can later load and use it without having to fit it again

In [16]:
import joblib
joblib.dump(knn_reg, 'k_nearest_model.pkl') 

['k_nearest_model.pkl']